<a href="https://colab.research.google.com/github/ahmedyahia01/ComfyUI-Colab/blob/main/ComfyUI_Installer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title الخلية 1: ربط Drive والتحقق من الإعدادات
import os
import sys
from google.colab import drive
from google.colab import userdata

print("--- 1. ربط Google Drive ---")
# سيطلب منك هذا الأمر السماح بالوصول إلى Google Drive الخاص بك
drive.mount('/content/drive')

print("\n--- 2. التحقق من مفتاح الوصول (Access Token) ---")
# سيقرأ هذا الكود مفتاح الوصول الذي حفظته في قسم الأسرار (Secrets)
try:
    hf_token = userdata.get('HF_TOKEN')
    if not hf_token or not hf_token.startswith("hf_"):
        print("❌ خطأ: مفتاح الوصول 'HF_TOKEN' غير موجود أو غير صالح.")
        print("   الرجاء التأكد من إضافته بشكل صحيح في قسم 🔑 Secrets على اليسار.")
        sys.exit("إيقاف التنفيذ: مفتاح الوصول مطلوب.")
    else:
        print("✅ تم العثور على مفتاح الوصول بنجاح.")
except Exception:
    print("❌ فشل في قراءة مفتاح الوصول. تأكد من أنك قمت بتفعيله لدفتر العمل هذا.")
    sys.exit("إيقاف التنفيذ: فشل قراءة المفتاح.")

# --- 3. إعداد المسارات الرئيسية ---
print("\n--- 3. إعداد المسارات ومجلد العمل ---")
BASE_DIR = "/content/drive/MyDrive/ComfyUI_Course_Project"
COMFYUI_DIR = os.path.join(BASE_DIR, "ComfyUI")
os.makedirs(BASE_DIR, exist_ok=True)
print(f"✅ سيتم إعداد كل شيء في المجلد: {BASE_DIR}")

print("\n🎉 اكتملت خلية الإعداد. أنت الآن جاهز لتشغيل خلية التثبيت والتحميل.")

In [ ]:
# @title الخلية 2: التثبيت والتحميل الشامل (تُشغل مرة واحدة وتستغرق وقتاً طويلاً)

import os
# --- التأكد من وجود المكتبات المطلوبة ---
print("--- 1/5: تثبيت وتحديث مكتبة Hugging Face ---")
!pip install -U huggingface_hub --quiet
from huggingface_hub import hf_hub_download
print("✅ المكتبة جاهزة.")

# --- التأكد من وجود ComfyUI ---
print("\n--- 2/5: التحقق من وجود ComfyUI ---")
if not os.path.exists(os.path.join(COMFYUI_DIR, "main.py")):
    print(f"📂 مجلد ComfyUI غير موجود، جاري تحميله...")
    !git clone --depth 1 https://github.com/comfyanonymous/ComfyUI.git "{COMFYUI_DIR}"
else:
    print("✅ مجلد ComfyUI موجود بالفعل.")

# --- تثبيت الإضافات (Custom Nodes) ---
print("\n--- 3/5: تثبيت الإضافات (Custom Nodes) ---")
nodes_dir = os.path.join(COMFYUI_DIR, "custom_nodes")
%cd $nodes_dir
custom_nodes_repos = {
    "ComfyUI-Manager": "https://github.com/ltdrdata/ComfyUI-Manager.git",
    "ComfyUI-Impact-Pack": "https://github.com/ltdrdata/ComfyUI-Impact-Pack.git",
}
for name, url in custom_nodes_repos.items():
    if not os.path.exists(name):
        print(f"  -> Installing {name}...")
        os.environ['GIT_TERMINAL_PROMPT'] = '0'
        !git clone {url}
    else:
        print(f"  -> {name} already exists.")

# --- تحميل النماذج الأساسية ---
print("\n--- 4/5: تحميل النماذج الأساسية (هذه الخطوة هي الأطول) ---")
checkpoints_dir = os.path.join(COMFYUI_DIR, "models", "checkpoints")
os.makedirs(checkpoints_dir, exist_ok=True)

# تحميل نموذج SD3 الشامل
print("  -> تحميل نموذج Stable Diffusion 3 الشامل...")
try:
    hf_hub_download(
        repo_id="stabilityai/stable-diffusion-3-medium",
        filename="sd3_medium_incl_clips.safetensors",
        local_dir=checkpoints_dir,
        local_dir_use_symlinks=False, # مهم جدًا لـ Google Drive
        token=hf_token
    )
    print("  -> 👍 تم تحميل نموذج SD3 بنجاح.")
except Exception as e:
    print(f"  -> ❌ فشل تحميل نموذج SD3. الخطأ: {e}")
    print("     تأكد من موافقتك على شروط النموذج في موقعه على Hugging Face.")

# --- تثبيت متطلبات بايثون ---
print("\n--- 5/5: تثبيت متطلبات بايثون للإضافات ---")
%cd $COMFYUI_DIR
!pip install -r requirements.txt --quiet
req_files = [os.path.join(root, name) for root, _, files in os.walk(nodes_dir) for name in files if name == 'requirements.txt']
for file in req_files:
    !pip install -r {file} --quiet

print("\n🎉🎉🎉 اكتملت عملية الإعداد والتحميل الطويلة! 🎉🎉🎉")
print("الآن يمكنك تشغيل الخلية الأخيرة لبدء ComfyUI.")

In [ ]:
# @title الخلية 3: تشغيل ComfyUI والحصول على الرابط

import time
import re

# --- تشغيل خادم ComfyUI في الخلفية ---
print("🚀 الخادم قيد التشغيل، الرجاء الانتظار 60 ثانية...")
%cd $COMFYUI_DIR
get_ipython().system_raw(f'python main.py --listen 0.0.0.0 --port 8188 --preview-method auto > comfy.log 2>&1 &')
time.sleep(60)

# --- محاولة إنشاء رابط عبر Cloudflared ---
print("\n🔄 محاولة إنشاء رابط عبر Cloudflared...")
if not os.path.exists("/content/cloudflared"):
    !wget -q -O /content/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
    !chmod +x /content/cloudflared

get_ipython().system_raw('/content/cloudflared tunnel --url http://localhost:8188 --no-autoupdate > /content/cloudflared.log 2>&1 &')
time.sleep(8)

# --- استخراج الرابط أو استخدام ngrok كبديل ---
link = None
try:
    with open("/content/cloudflared.log", "r") as f:
        content = f.read()
        match = re.search(r"https://[A-Za-z0-9_-]+\.trycloudflare\.com", content)
        if match:
            link = match.group(0)
except FileNotFoundError:
    pass

if link:
    print("\n" + "="*60)
    print("✅ رابط ComfyUI عبر Cloudflared:", link)
    print("="*60)
else:
    print("\n⚠️ فشل Cloudflared، جاري محاولة استخدام ngrok كبديل...")
    !pip install pyngrok --quiet
    from pyngrok import ngrok
    try:
        public_url = ngrok.connect(8188)
        print("\n" + "="*60)
        print("✅ رابط ComfyUI عبر ngrok:", public_url)
        print("="*60)
    except Exception as e:
        print(f"\n❌ فشل ngrok أيضًا. الخطأ: {e}")